In [ ]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import random
import os

train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kc_house_data.csv')

In [ ]:
def my_seed_everywhere(seed: int = 42):
    random.seed(seed) # random
    np.random.seed(seed) # np
    os.environ["PYTHONHASHSEED"] = str(seed) # os
    tf.random.set_seed(seed) # tensorflow

my_seed = 42
my_seed_everywhere(my_seed)

In [ ]:
train.shape

(21613, 21)

In [ ]:
train.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


#### 해당 구역의 데이터 같아 보임
- 위경도, 우편주소, id, 거래 날짜 버리고 (DNN에서)
- 나머지 다 버리고 거래 날짜와 거래 가격으로만 (RNN)

In [ ]:
train = train.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis=1)

In [ ]:
X = train.drop('price', axis=1)
y = train['price']

In [ ]:
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(300, activation='relu', input_shape=(X.shape[1],)),
        tf.keras.layers.Dense(300, activation='relu'),
        tf.keras.layers.Dense(300, activation='relu'),
        tf.keras.layers.Dense(300, activation='relu'),
        tf.keras.layers.Dense(300, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    optim = tf.keras.optimizers.RMSprop(learning_rate=0.001)
    model.compile(optimizer=optim, loss='mse', metrics=['mse'])
    return model


In [ ]:
k = 10
num_val = len(X) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold #', i)

    X_val = X[i * num_val: (i + 1) * num_val]
    y_val = y[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate([X[:i * num_val], X[(i + 1) * num_val:]], axis=0)
    y_train_part = np.concatenate([y[:i * num_val], y[(i + 1) * num_val:]], axis=0)

    model = build_model()
    model.fit(X_train_part, y_train_part, epochs=num_epochs, verbose=1)

    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    all_scores.append(val_acc)


processing fold # 0
Epoch 1/100
608/608 [==============================] - 5s 4ms/step - loss: 103073603584.0000 - mse: 103073603584.0000
Epoch 2/100
608/608 [==============================] - 3s 4ms/step - loss: 71980253184.0000 - mse: 71980253184.0000
Epoch 3/100
608/608 [==============================] - 3s 5ms/step - loss: 68702924800.0000 - mse: 68702924800.0000
Epoch 4/100
608/608 [==============================] - 2s 4ms/step - loss: 63003742208.0000 - mse: 63003742208.0000
Epoch 5/100
608/608 [==============================] - 2s 4ms/step - loss: 59805458432.0000 - mse: 59805458432.0000
Epoch 6/100
608/608 [==============================] - 2s 4ms/step - loss: 58196606976.0000 - mse: 58196606976.0000
Epoch 7/100
608/608 [==============================] - 2s 4ms/step - loss: 56610058240.0000 - mse: 56610058240.0000
Epoch 8/100
608/608 [==============================] - 3s 4ms/step - loss: 55860420608.0000 - mse: 55860420608.0000
Epoch 9/100
608/608 [=============================

In [ ]:


mean_scores_sqrt = np.sqrt(mean_scores)
mean_scores_sqrt


222311.196313636

### 결과1

In [ ]:
all_scores #-> 이담

[63734149120.0, 56770453504.0, 50354786304.0, 58519318528.0, 62529667072.0]

In [ ]:
mean_scores = np.array(all_scores).mean()
mean_scores

58643913113.6

In [ ]:


mean_scores_sqrt = np.sqrt(mean_scores)
mean_scores_sqrt


242165.05345239225

### 결과2

In [ ]:
# def build_model():
#     model = tf.keras.Sequential([
#         tf.keras.layers.Dense(300, activation='relu', input_shape=(X.shape[1],)),
#         tf.keras.layers.Dense(300, activation='relu'),
#         tf.keras.layers.Dense(300, activation='relu'),
#         tf.keras.layers.Dense(300, activation='relu'),
#         tf.keras.layers.Dense(1)
#     ])
#     optim = tf.keras.optimizers.RMSprop(learning_rate=0.001)
#     model.compile(optimizer=optim, loss='mse', metrics=['mse'])
#     return model

In [ ]:
all_scores

[63734149120.0, 56770453504.0, 50354786304.0, 58519318528.0, 62529667072.0]

In [ ]:
mean_scores = np.array(all_scores).mean()
mean_scores

58381674905.6

In [ ]:


mean_scores_sqrt = np.sqrt(mean_scores)
mean_scores_sqrt


241623.001607049

### 결과 3

In [ ]:
all_scores

    #     tf.keras.layers.Dense(300, activation='relu', input_shape=(X.shape[1],)),
    #     tf.keras.layers.Dense(300, activation='relu'),
    #     tf.keras.layers.Dense(300, activation='relu'),
    #     tf.keras.layers.Dense(300, activation='relu'),
    #     tf.keras.layers.Dense(1)
    # ])

[45815521280.0,
 56106340352.0,
 43938156544.0,
 55822589952.0,
 38770073600.0,
 42501619712.0,
 47608692736.0,
 49857376256.0,
 50594664448.0,
 63207645184.0]

In [ ]:
mean_scores = np.array(all_scores).mean()
mean_scores

49422268006.4

In [ ]:


mean_scores_sqrt = np.sqrt(mean_scores)
mean_scores_sqrt


222311.196313636

### 최종

### 결과 3이 제일 좋음

In [ ]:
all_scores

[46465302528.0,
 55384907776.0,
 43545526272.0,
 51842654208.0,
 39456755712.0,
 42086072320.0,
 48803794944.0,
 48138199040.0,
 48720535552.0,
 62545686528.0]

In [ ]:
mean_scores = np.array(all_scores).mean()
mean_scores

48698943488.0